# Rodrigo Barraza's Inscriptions: Blip 2 Mass Captioning
Large RAM and VRAM is required to load the larger models. RAM should be at least 24-32GB with 64GB being optimal. VRAM should be at least 16GB or more.

In [1]:
!pip3 install salesforce-lavis --upgrade
!pip3 install validators


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import sys
import validators
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

/home/rodrigo/miniconda3/envs/blip2demo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load BLIP2 captioning model

In [3]:
# setup device to use
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
# we associate a model with its preprocessors to make it easier for inference.
model, vis_processors, _ = load_model_and_preprocess(
    # name="blip2_opt", model_type="pretrain_opt2.7b", is_eval=True, device=device
    # name="blip2_opt", model_type="pretrain_opt6.7b", is_eval=True, device=device
    # name="blip2_opt", model_type="caption_coco_opt2.7b", is_eval=True, device=device
    name="blip2_opt", model_type="caption_coco_opt6.7b", is_eval=True, device=device
    # name="blip2_t5", model_type="pretrain_flant5xl", is_eval=True, device=device
    # name="blip2_t5", model_type="caption_coco_flant5xl", is_eval=True, device=device
    # This next model is one scary devil in terms of size. It requires at least 32GB of VRAM to run, and will not load on 3090s or 4090s.
    # name="blip2_t5", model_type="pretrain_flant5xxl", is_eval=True, device=device
)

vis_processors.keys()

Position interpolate from 16x16 to 26x26


Loading checkpoint shards: 100%|██████████| 2/2 [00:33<00:00, 16.87s/it]


dict_keys(['train', 'eval'])

#### Auto Caption

In [8]:
import os
from pathlib import Path
from IPython.display import clear_output

# Start of Options
imagesDirectory = "/mnt/d/dataset/_characters fictional/dexter mcpherson"
useFoldersAsTokens = True # Append the folder names to the beginning of the caption
tokensStartOrEnd = 'start' # end or start
minTokenLength = 5 # The amount of minimum tokens to generate
maxTokenLength = 72 # The maximum amount of tokens to generate
numberOfCaptions = 1 # How many captions to generate
useNucleusSampling = False
repetitionPenalty = 1
# End of Options


def process_images(dirpath):
    suspects = os.listdir(dirpath) 
    imageSuspects = [filename for filename in suspects if filename.lower().endswith((".jpg", ".png", ".jpeg", ".webp", "gif"))]

    for suspectIndex in range(len(imageSuspects)):
        caption = ""
        suspectsLength = len(imageSuspects) + 1
        print(f"{suspectsLength - suspectIndex}/{suspectsLength}")
        # Load Image
        imagePath = imageSuspects[suspectIndex]
        imageFilePath = dirpath + "/" + imagePath
        textFilePath = Path(imageFilePath).with_suffix('.txt')
        # If file doesn't exist:
        if not os.path.exists(textFilePath):
            rawImage = Image.open(imageFilePath).convert('RGB')
            # display(rawImage)
            image = vis_processors["eval"](rawImage).unsqueeze(0).to(device)
            imageCaption = model.generate({"image": image}, min_length=minTokenLength, max_length=maxTokenLength, use_nucleus_sampling=useNucleusSampling, num_captions=numberOfCaptions, repetition_penalty=repetitionPenalty)
            if useFoldersAsTokens:
                relpath = os.path.relpath(dirpath, imagesDirectory)
                relpath_parts = relpath.split(os.sep)
                if tokensStartOrEnd == 'end':
                    caption += imageCaption[0]
                    for part in relpath_parts:
                        if "_" not in part and part != ".":
                            imageCaptionSet = set(imageCaption[0].split())
                            if part not in imageCaptionSet:
                                    caption += ", " + part
                else:
                    caption = ""
                    for part in relpath_parts:
                        if "_" not in part and part != ".":
                            imageCaptionSet = set(imageCaption[0].split())
                            if part not in imageCaptionSet:
                                caption += part + ", "
                    caption += imageCaption[0]
            else:
                caption = imageCaption[0]
            # Remove periods
            caption.replace('.', '')
            # Save Caption as .txt file
            with open(textFilePath, 'w+') as f:
                f.write(caption)
        clear_output(wait=True)
        print(caption)
        print(imageFilePath)

# Iterate through directories inside directories
for dirpath, dirnames, filenames in os.walk(imagesDirectory):
    process_images(dirpath)

funko, the simpsons pop vinyl figure
/mnt/d/dataset/_characters fictional/dexter mcpherson/funko/dexterslab39.jpeg
